# Tabulating and saving correlations functions with TabCorr

## particle displacement done in separate script in `baryonic_effects/baryonification/displace_MDPL2_particles.py`  

In [1]:
import numpy as np
from matplotlib import cm
from matplotlib import colors
import matplotlib.pyplot as plt
from halotools.sim_manager import CachedHaloCatalog
from halotools.mock_observables import wp, delta_sigma
from halotools.empirical_models import PrebuiltHodModelFactory
from tabcorr import TabCorr

import pandas as pd
from halotools.sim_manager import UserSuppliedHaloCatalog, UserSuppliedPtclCatalog,FakeSim
from halotools.mock_observables import delta_sigma, wp, return_xyz_formatted_array
from halotools.empirical_models import NFWProfile
from halotools.empirical_models import PrebuiltHodModelFactory, HodModelFactory
from halotools.empirical_models import AssembiasZheng07Cens, AssembiasZheng07Sats, TrivialPhaseSpace, NFWPhaseSpace
from halotools.utils import add_halo_hostid

import time

In [2]:
def create_halo_and_particle_catalogs_for_halotools(halo_catalog_path, particle_catalog_path):
    halo_df = pd.read_csv(halo_catalog_path)
    particles_df = pd.read_csv(particle_catalog_path, delimiter =' +',  names=['x','y','z'], engine='python')
    print('Files read.')
    
    ptcl_x = particles_df['x'].values
    ptcl_y = particles_df['y'].values
    ptcl_z = particles_df['z'].values

    particle_mass = 1.51e9
    num_ptcl_per_dim = 3840
    x = halo_df['x'].values
    y = halo_df['y'].values
    z = halo_df['z'].values
    vx = halo_df['vx'].values
    vy = halo_df['vy'].values
    vz = halo_df['vz'].values
    mass = halo_df['Mvir'].values
    radius = halo_df['Rvir'].values/1e3 #convert to Mpc
    ids = np.arange(0, len(halo_df))
    upid = halo_df['upId'].values
    simname = 'MDPL2'
    
    #get concentrations
#     nfw = NFWProfile(redshift=redshift, cosmology = Planck15, mdef = 'vir', conc_mass_model = 'dutton_maccio14')
#     model_conc = nfw.conc_NFWmodel(prim_haloprop = mass)
    concentrations = halo_df['Rvir'].values / halo_df['Rs'].values
    
    print('Creating catalogs...')
    particle_catalog = UserSuppliedPtclCatalog(x = ptcl_x, y = ptcl_y, z = ptcl_z, Lbox = Lbox, particle_mass = particle_mass,
                                  redshift = redshift)
    halo_catalog = UserSuppliedHaloCatalog(user_supplied_ptclcat = particle_catalog, redshift = redshift, simname = simname,
                                       Lbox = Lbox, particle_mass = particle_mass, num_ptcl_per_dim =num_ptcl_per_dim,
                                       halo_x = x, halo_y = y, halo_z = z,
                                       halo_vx = vx, halo_vy = vy, halo_vz = vz, 
                                       halo_id = ids, halo_mvir = mass, halo_rvir = radius,
                                       halo_nfw_conc = concentrations, halo_upid = upid )
    
    #add hostid
    add_halo_hostid(halo_catalog.halo_table)
    
    return halo_catalog, particle_catalog

In [3]:
data_directory = '/Users/fardila/Documents/Data/baryonic_effects/CMASS/'
queried_halo_cat_file = 'halo_catalogs/mdpl2_hlist_0.65650_Mvir11.2.csv'
test_halo_cat_file = 'halo_catalogs/test.csv'
### "row_id","Mvir","Rvir","M200c","M500c","x","y","z","scale"
# full_halo_cat_file = 'halo_catalogs/cut_halo_df.pkl'
particle_cat_file = 'particle_catalogs/mdpl2_particles_0.6565_10m.dat'
test_particle_cat_file = 'particle_catalogs/test.dat'
### "x","y","z"
displacedA_particle_cat_file = 'particle_catalogs/MDPL2_bfc_particles_A.out'
displacedB_particle_cat_file = 'particle_catalogs/MDPL2_bfc_particles_B.out'
displacedC_particle_cat_file = 'particle_catalogs/MDPL2_bfc_particles_C.out'

In [4]:
redshift = (1./0.65650)-1. #z=0.523
Lbox = 1000. #Mpc/h

In [5]:
halo_catalog, particle_catalog = create_halo_and_particle_catalogs_for_halotools(data_directory+queried_halo_cat_file,
                                                                                 data_directory+particle_cat_file)

Files read.
Creating catalogs...


In [9]:
halo_catalog, displaced_particle_catalog = create_halo_and_particle_catalogs_for_halotools(data_directory+queried_halo_cat_file,
                                                                                 data_directory+displacedA_particle_cat_file)


Files read.
Creating catalogs...


# Tabulate DS for MDPL2 (before baryonification) 

In [6]:
particle_masses = halo_catalog.particle_mass
period=halo_catalog.Lbox
downsampling_factor = (halo_catalog.num_ptcl_per_dim**3)/float(len(particle_catalog.ptcl_table))

In [7]:
x = particle_catalog.ptcl_table['x']
y = particle_catalog.ptcl_table['y']
z = particle_catalog.ptcl_table['z']

particle_positions = return_xyz_formatted_array(x, y, z, period=period)

In [8]:
time1 = time.time()


# First, we tabulate the correlation functions in the halo catalog.
rp_bins = np.logspace(-1, 1, 20)

halotab = TabCorr.tabulate(halo_catalog, delta_sigma, particle_positions, rp_bins = rp_bins,
                           mode ='cross', period = period, particle_masses = particle_masses,
                           downsampling_factor = downsampling_factor )

# We can save the result for later use.
halotab.write('mdpl2_tabCorr_DS.hdf5')

print('{0} seconds'.format(time.time() - time1)) 

/Users/fardila/anaconda2/envs/baryonic_effects/lib/python3.7/site-packages/halotools/empirical_models/phase_space_models/analytic_models/monte_carlo_helpers.py:205: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.rad_prof_func_table_indices[digitized_param_list]
/Users/fardila/anaconda2/envs/baryonic_effects/lib/python3.7/site-packages/halotools/empirical_models/phase_space_models/analytic_models/monte_carlo_helpers.py:522: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.rad_prof_func_table_indices[digitized_param_list]


Number of tracer particles: 73510379
10594.626524925232 seconds


# Tabulate DS for MDPL2 (after baryonification) 

In [10]:
particle_masses = halo_catalog.particle_mass
period=halo_catalog.Lbox
downsampling_factor = (halo_catalog.num_ptcl_per_dim**3)/float(len(displaced_particle_catalog.ptcl_table))

In [11]:
displaced_x = displaced_particle_catalog.ptcl_table['x']
displaced_y = displaced_particle_catalog.ptcl_table['y']
displaced_z = displaced_particle_catalog.ptcl_table['z']

displaced_particle_positions = return_xyz_formatted_array(displaced_x, displaced_y, displaced_z, period=period)

In [12]:
time1 = time.time()


# First, we tabulate the correlation functions in the halo catalog.
rp_bins = np.logspace(-1, 1, 20)

halotab = TabCorr.tabulate(halo_catalog, delta_sigma, displaced_particle_positions, rp_bins = rp_bins,
                           mode ='cross', period = period, particle_masses = particle_masses,
                           downsampling_factor = downsampling_factor )

# We can save the result for later use.
halotab.write('mdpl2+baryonification_tabCorr_DS.hdf5')

print('{0} seconds'.format(time.time() - time1)) 

/Users/fardila/anaconda2/envs/baryonic_effects/lib/python3.7/site-packages/halotools/empirical_models/phase_space_models/analytic_models/monte_carlo_helpers.py:205: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.rad_prof_func_table_indices[digitized_param_list]
/Users/fardila/anaconda2/envs/baryonic_effects/lib/python3.7/site-packages/halotools/empirical_models/phase_space_models/analytic_models/monte_carlo_helpers.py:522: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.rad_prof_func_table_indices[digitized_param_list]


Number of tracer particles: 73522971
10647.422044038773 seconds
